In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np

import pandas as pd
from sklearn.metrics import roc_curve, auc
from tqdm import tqdm
import zlib

import torch
import torch.nn.functional as F
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "state-spaces/mamba-1.4b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)





In [ ]:
def get_metrics(scores, labels):
    fpr_list, tpr_list, thresholds = roc_curve(labels, scores)
    auroc = auc(fpr_list, tpr_list)
    fpr95 = fpr_list[np.where(tpr_list >= 0.95)[0][0]]
    tpr05 = tpr_list[np.where(fpr_list <= 0.05)[0][-1]]
    return auroc, fpr95, tpr05
def format_metrics(metrics):
  mean_auroc = metrics['mean_auroc']
  lower_bound, upper_bound = metrics['auroc_ci']
  return f'{mean_auroc:.3f} ( {lower_bound:.3f} , {upper_bound:.3f} )'

def get_and_format_metrics(scores, labels, n_bootstraps=1000, alpha=0.95):
  metrics = get_metrics_with_confidence_interval(scores, labels,n_bootstraps,alpha)
  return format_metrics(metrics)


def get_metrics_with_confidence_interval(scores, labels, n_bootstraps=1000, alpha=0.95):
        # Convert labels and scores to numpy arrays if they are lists
        labels = np.array(labels)
        scores = np.array(scores)

        # Calculate initial AUROC, FPR at 95% TPR, and TPR at 5% FPR
        fpr_list, tpr_list, thresholds = roc_curve(labels, scores)
        auroc = auc(fpr_list, tpr_list)
        fpr95 = fpr_list[np.where(tpr_list >= 0.95)[0][0]]
        tpr05 = tpr_list[np.where(fpr_list <= 0.05)[0][-1]]

        bootstrapped_scores = []

        for i in range(n_bootstraps):
            # Resample with replacement from the original data
            indices = resample(np.arange(len(labels)), replace=True, n_samples=len(labels))
            if len(np.unique(labels[indices])) < 2:
                # Skip this iteration if the resampled data does not have at least two classes
                continue

            score = roc_auc_score(labels[indices], scores[indices])
            bootstrapped_scores.append(score)

        sorted_scores = np.sort(bootstrapped_scores)

        # Calculate the lower and upper percentiles for the confidence interval
        lower_bound = np.percentile(sorted_scores, (1 - alpha) / 2 * 100)
        upper_bound = np.percentile(sorted_scores, (1 + alpha) / 2 * 100)

        # Calculate mean AUROC from bootstrapped scores
        mean_auroc = np.mean(bootstrapped_scores)

        return {
            "auroc": auroc,
            "mean_auroc": mean_auroc,
            "auroc_ci": (lower_bound, upper_bound),
            "fpr95": fpr95,
            "tpr05": tpr05
        }
def calculate_min_k_prob_plusplus(model,tokenizer,texts,ratios):
  results = []
  for text in tqdm(texts):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
    input_ids = input_ids.to(model.device)
    with torch.no_grad():
      outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    ll = -loss.item() # log-likelihood
    # assuming the score is larger for training data
    # and smaller for non-training data
    # this is why sometimes there is a negative sign in front of the score
    input_ids = input_ids[0][1:].unsqueeze(-1)
    probs = F.softmax(logits[0, :-1], dim=-1)
    log_probs = F.log_softmax(logits[0, :-1], dim=-1)
    token_log_probs = log_probs.gather(dim=-1, index=input_ids).squeeze(-1)
    mu = (probs * log_probs).sum(-1)
    sigma = (probs * torch.square(log_probs)).sum(-1) - torch.square(mu)
    mink_plus = (token_log_probs - mu) / sigma.sqrt()
    result = {}
    for ratio in ratios:
      k_length = int(len(mink_plus) * ratio)
      topk = np.sort(mink_plus.cpu())[:k_length]
      result[f'mink++_{ratio}_mean'] = np.mean(topk).item()
      result[f'mink++_{ratio}_median'] = np.median(topk).item()
    results.append(result)
      # print(f'mink++_{ratio}_mean',np.mean(topk).item())
      # print(f'mink++_{ratio}_median',np.median(topk).item())
  return results


In [ ]:
%pip install datasets
import pandas as pd
import datasets
dataset = datasets.load_dataset('avduarte333/BookTection')
df = dataset['train'].to_pandas()
df_label_0 = df[df['Label'] == 0]
df_label_1 = df[df['Label'] == 1]

df_label_0 = df_label_0.sample(n=300, random_state=42)
df_label_1 = df_label_1.sample(n=300, random_state=42)

df = pd.concat([df_label_0, df_label_1])



df
df.to_csv('bookTection_sample.csv')

Generating train split:   0%|          | 0/16414 [00:00<?, ? examples/s]

In [ ]:
df = pd.read_csv('bookTection_sample.csv')

In [ ]:
texts = df['Example_A'].to_list()
labels = df['Label'].to_list()

In [ ]:
texts[0]

'Please, I wantI want these children to go to school. Its their only chance in life. I dont have any property Im going to leave them. I have nothing else to give them that can You think I am happy to see you suffer like this? Why do you think I am angry with your husband? God has not blessed this family with wealth, but were you ever hungry? Look at the wells in your neck. If he knew he could not take care of you, why did he marry you? You too look at what you are wearingwere you not better dressed when you lived here? y nil looked down at her washed-out ankara dress. The hem had come apart, and if you stared long enough, you could see that it had been patched at the hip. Let me stop talking about your husband before this becomes another fight. Okay sir. But, y nil, I cant afford to give you more than one thousand five hundred naira. Me too, I just finished paying school fees last week. My purse is still recovering from that. y nil leaned forward. Surely shed misheard. You just finishe

In [ ]:
labels[32],labels[12]

(0, 0)

In [ ]:
len(texts),len(labels)

(600, 600)

In [ ]:
ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
results = calculate_min_k_prob_plusplus(model,tokenizer,texts,ratios)

100%|██████████| 600/600 [56:07<00:00,  5.61s/it]


In [ ]:
def get_metrics(scores, labels):
    fpr_list, tpr_list, thresholds = roc_curve(labels, scores)
    auroc = auc(fpr_list, tpr_list)
    fpr95 = fpr_list[np.where(tpr_list >= 0.95)[0][0]]
    tpr05 = tpr_list[np.where(fpr_list <= 0.05)[0][-1]]
    return auroc, fpr95, tpr05

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df['snippet'] = texts
df['label'] = labels
df

,mink++_0.1_mean,mink++_0.1_median,mink++_0.2_mean,mink++_0.2_median,mink++_0.3_mean,mink++_0.3_median,mink++_0.4_mean,mink++_0.4_median,mink++_0.5_mean,mink++_0.5_median,...,mink++_0.7_mean,mink++_0.7_median,mink++_0.8_mean,mink++_0.8_median,mink++_0.9_mean,mink++_0.9_median,mink++_1.0_mean,mink++_1.0_median,snippet,label
0,-2.514396,-2.239030,-1.711973,-1.269656,-1.231199,-0.960675,-0.917178,-0.536769,-0.685779,-0.235066,...,-0.363864,0.061621,-0.235968,0.140630,-0.123568,0.237934,-0.009813,0.317938,"Please, I wantI want these children to go to s...",0
1,-1.944683,-1.429702,-1.325487,-0.953395,-0.964939,-0.714689,-0.699887,-0.446278,-0.506490,-0.224560,...,-0.242339,0.116063,-0.135796,0.191633,-0.038541,0.273594,0.067800,0.320173,"He patted the bed next to him and I sat down, ...",0
2,-2.092384,-2.003510,-1.510507,-1.380110,-1.166979,-0.833384,-0.837878,-0.708568,-0.624163,-0.485950,...,-0.342087,0.117868,-0.232573,0.157863,-0.122958,0.181646,-0.017526,0.217467,Im here because somebody has to bear witness. ...,0
3,-2.500147,-2.158324,-1.794055,-1.561654,-1.369106,-1.051875,-1.058328,-0.746395,-0.809507,-0.521481,...,-0.455883,-0.086351,-0.324428,0.091881,-0.203918,0.164783,-0.082138,0.278103,Her eyelash extensions mean if I look close en...,0
4,-2.003646,-1.905039,-1.346438,-1.151779,-0.970430,-0.648282,-0.714773,-0.464438,-0.525797,-0.177049,...,-0.254526,0.064323,-0.143199,0.161016,-0.044360,0.231202,0.056990,0.318740,An act that boggles my mind. This is my father...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-2.718343,-2.573054,-1.958414,-1.723387,-1.512079,-1.067651,-1.163465,-0.870128,-0.901351,-0.608611,...,-0.519921,-0.085608,-0.377657,0.043389,-0.247858,0.122102,-0.113410,0.260845,“He might have been tortured. Or persuaded. My...,1
596,-2.396000,-2.337178,-1.663090,-1.434688,-1.227146,-0.920908,-0.919438,-0.548048,-0.690132,-0.370667,...,-0.358484,0.032188,-0.233434,0.100178,-0.121777,0.224026,-0.009724,0.329797,Such unequal attachments had led to marriage; ...,1
597,-2.515041,-2.315437,-1.718944,-1.691028,-1.237079,-0.849421,-0.909986,-0.521508,-0.671664,-0.267493,...,-0.367937,0.060528,-0.252611,0.154769,-0.149913,0.234076,-0.039256,0.301952,"""Hoax all the same,"" said Marvel. ""I know the ...",1
598,-2.485605,-2.379416,-1.701803,-1.355887,-1.255822,-0.850124,-0.949651,-0.661047,-0.723417,-0.341137,...,-0.395362,0.003286,-0.267981,0.085855,-0.155662,0.181087,-0.034774,0.273186,But his hand automatically kept on making the ...,1


In [ ]:
df = pd.read_csv('mamba_mink++_bookTection_results.csv')
ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

df

,Unnamed: 0,mink++_0.1_mean,mink++_0.1_median,mink++_0.2_mean,mink++_0.2_median,mink++_0.3_mean,mink++_0.3_median,mink++_0.4_mean,mink++_0.4_median,mink++_0.5_mean,...,mink++_0.7_mean,mink++_0.7_median,mink++_0.8_mean,mink++_0.8_median,mink++_0.9_mean,mink++_0.9_median,mink++_1.0_mean,mink++_1.0_median,snippet,label
0,0,-2.514396,-2.239030,-1.711973,-1.269656,-1.231199,-0.960675,-0.917178,-0.536769,-0.685779,...,-0.363864,0.061621,-0.235968,0.140630,-0.123568,0.237934,-0.009813,0.317938,"Please, I wantI want these children to go to s...",0
1,1,-1.944683,-1.429702,-1.325487,-0.953395,-0.964939,-0.714689,-0.699887,-0.446278,-0.506490,...,-0.242339,0.116063,-0.135796,0.191633,-0.038541,0.273594,0.067800,0.320173,"He patted the bed next to him and I sat down, ...",0
2,2,-2.092384,-2.003510,-1.510507,-1.380110,-1.166979,-0.833384,-0.837878,-0.708568,-0.624163,...,-0.342087,0.117868,-0.232573,0.157863,-0.122958,0.181646,-0.017526,0.217467,Im here because somebody has to bear witness. ...,0
3,3,-2.500147,-2.158324,-1.794055,-1.561654,-1.369106,-1.051875,-1.058328,-0.746395,-0.809507,...,-0.455883,-0.086351,-0.324428,0.091881,-0.203918,0.164783,-0.082138,0.278103,Her eyelash extensions mean if I look close en...,0
4,4,-2.003646,-1.905039,-1.346438,-1.151779,-0.970430,-0.648282,-0.714773,-0.464438,-0.525797,...,-0.254526,0.064323,-0.143199,0.161016,-0.044360,0.231202,0.056990,0.318740,An act that boggles my mind. This is my father...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,595,-2.718343,-2.573054,-1.958414,-1.723387,-1.512079,-1.067651,-1.163465,-0.870128,-0.901351,...,-0.519921,-0.085608,-0.377657,0.043389,-0.247858,0.122102,-0.113410,0.260845,“He might have been tortured. Or persuaded. My...,1
596,596,-2.396000,-2.337178,-1.663090,-1.434688,-1.227146,-0.920908,-0.919438,-0.548048,-0.690132,...,-0.358484,0.032188,-0.233434,0.100178,-0.121777,0.224026,-0.009724,0.329797,Such unequal attachments had led to marriage; ...,1
597,597,-2.515041,-2.315437,-1.718944,-1.691028,-1.237079,-0.849421,-0.909986,-0.521508,-0.671664,...,-0.367937,0.060528,-0.252611,0.154769,-0.149913,0.234076,-0.039256,0.301952,"""Hoax all the same,"" said Marvel. ""I know the ...",1
598,598,-2.485605,-2.379416,-1.701803,-1.355887,-1.255822,-0.850124,-0.949651,-0.661047,-0.723417,...,-0.395362,0.003286,-0.267981,0.085855,-0.155662,0.181087,-0.034774,0.273186,But his hand automatically kept on making the ...,1


In [ ]:
cleaned = df.dropna()

In [ ]:
len(cleaned)

600

In [ ]:
for avg_type in ['mean']:
  for ratio in ratios:
    column = f'mink++_{ratio}_{avg_type}'
    print(f'{column}')
    print(get_and_format_metrics(df[column].to_list(),df['label'].to_list()))
    print()

mink++_0.1_mean
0.612 ( 0.571 , 0.655 )

mink++_0.2_mean
0.620 ( 0.577 , 0.665 )

mink++_0.3_mean
0.623 ( 0.578 , 0.668 )

mink++_0.4_mean
0.621 ( 0.574 , 0.664 )

mink++_0.5_mean
0.618 ( 0.574 , 0.659 )

mink++_0.6_mean
0.616 ( 0.570 , 0.660 )

mink++_0.7_mean
0.617 ( 0.568 , 0.661 )

mink++_0.8_mean
0.615 ( 0.567 , 0.660 )

mink++_0.9_mean
0.617 ( 0.572 , 0.661 )

mink++_1.0_mean
0.616 ( 0.571 , 0.661 )



In [ ]:
df.to_csv('mamba_mink++_bookTection_results.csv')